Chargement des Librairies

In [1]:
#Librairies
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()
from sklearn.cluster import AgglomerativeClustering, KMeans

Chargement des CSV et affichage des infos

In [2]:
#GenomeScores qui donne la relevance par tagIg et par movieId

dfgenome=pd.read_csv("../data/genome-scores.csv" )
display(dfgenome.head(5))
display(dfgenome.info())

,movieId,tagId,relevance
0,1,1,0.02500
1,1,2,0.02500
2,1,3,0.05775
3,1,4,0.09675
4,1,5,0.14675


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11709768 entries, 0 to 11709767
Data columns (total 3 columns):
 #   Column     Dtype  
---  ------     -----  
 0   movieId    int64  
 1   tagId      int64  
 2   relevance  float64
dtypes: float64(1), int64(2)
memory usage: 268.0 MB


None

In [3]:
#GenomeTags qui donne la correspondance des Tags avec leur TagId
dftags=pd.read_csv("../data/genome-tags.csv", index_col="tagId")
display(dftags.head(5))
display(dftags.info())

,tag
tagId,
1,007
2,007 (series)
3,18th century
4,1920s
5,1930s


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1128 entries, 1 to 1128
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tag     1128 non-null   object
dtypes: object(1)
memory usage: 17.6+ KB


None

In [4]:
#movies  qui donne la correspondance des titre des films, leur genre avec leur movieId
dfmovies=pd.read_csv("../data/movies.csv", sep=',', index_col="movieId")
display(dfmovies.info())
display(dfmovies.head(5))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27278 entries, 1 to 131262
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   27278 non-null  object
 1   genres  27278 non-null  object
dtypes: object(2)
memory usage: 639.3+ KB


None

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [5]:
#ratings  qui recence les notes des utilisateurs avec la correspondance des Id des utilisateurs, des Id des films,
#la note (rating) et l'instant de la notation (timestamp)

dfratings=pd.read_csv("../data/ratings.csv", sep=',')
display(dfratings.info())
display(dfratings.head(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000263 entries, 0 to 20000262
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 610.4 MB


None

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


Adding a column with the year of the movie in dfmovies tab

In [6]:
#selection of the end of the title
dfmovies["year"]=dfmovies["title"].str.slice(start=-6)
#slit of "()"
dfmovies["year"]=dfmovies["year"].str.split(pat="(",expand=True)[1].str.split(pat=")",expand=True)[0]

#dropna of the not-conform title
dfmovies=dfmovies.dropna()

#conversion to int of the year
dfmovies["year"]=dfmovies["year"].astype(int)
dfmovies.head()

,title,genres,year
movieId,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
2,Jumanji (1995),Adventure|Children|Fantasy,1995
3,Grumpier Old Men (1995),Comedy|Romance,1995
4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
5,Father of the Bride Part II (1995),Comedy,1995


We check that the year is well converted into int

In [7]:
#info on df movies
dfmovies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27049 entries, 1 to 131262
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   27049 non-null  object
 1   genres  27049 non-null  object
 2   year    27049 non-null  int32 
dtypes: int32(1), object(2)
memory usage: 739.6+ KB


We use the year to insert a coefficient to prefer recent movies

In [8]:
#creation of the coefficient from movie ages
dfmovies["coef_age"]=np.log((dfmovies["year"]-dfmovies["year"].min())+1)

In [9]:
#visualisation of the dataframe dfmovies with the nexw coefficient of ages
dfmovies.head(60)

,title,genres,year,coef_age
movieId,,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,4.653960
2,Jumanji (1995),Adventure|Children|Fantasy,1995,4.653960
3,Grumpier Old Men (1995),Comedy|Romance,1995,4.653960
4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995,4.653960
5,Father of the Bride Part II (1995),Comedy,1995,4.653960
6,Heat (1995),Action|Crime|Thriller,1995,4.653960
7,Sabrina (1995),Comedy|Romance,1995,4.653960
8,Tom and Huck (1995),Adventure|Children,1995,4.653960
9,Sudden Death (1995),Action,1995,4.653960


In [43]:
dfmovies.to_csv('../data/dfmovies_coef_age.csv')

Selection of the 50 most relevant tagId

In [10]:
#groupby tagId and sum to compute the total relevance by tagId. Desascending sorting to keep only the 50 most relevant. 
toprelevance = dfgenome.groupby('tagId').agg(sum).sort_values('relevance',ascending=False).head(50).index


In [11]:
#selection in the main dataframe dfgenome from the grouped dataframe selection above.
genome_top = dfgenome[dfgenome['tagId'].isin(toprelevance)]

In [12]:
#check of the tab from the selection
display(genome_top.head(5))
display(genome_top.info())

,movieId,tagId,relevance
20,1,21,0.33025
21,1,22,0.28250
168,1,169,0.20525
187,1,188,0.48600
194,1,195,0.59000


<class 'pandas.core.frame.DataFrame'>
Int64Index: 519050 entries, 20 to 11709743
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   movieId    519050 non-null  int64  
 1   tagId      519050 non-null  int64  
 2   relevance  519050 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 15.8 MB


None

In [13]:
#compression of the selection table by pivot method.
genome_top=pd.pivot_table(genome_top, index="movieId",columns="tagId")

In [14]:
#check of the tab from the pivot method
display(genome_top.head(5))
display(genome_top.info())

relevance                                                       \
tagId        21       22       169     188      195      203      212    
movieId                                                                  
1         0.33025  0.28250  0.20525  0.4860  0.59000  0.92625  0.12350   
2         0.45100  0.60200  0.22825  0.4865  0.62600  0.87025  0.12975   
3         0.28000  0.20775  0.29950  0.4405  0.53800  0.34850  0.13750   
4         0.49875  0.44650  0.31100  0.4185  0.26125  0.24075  0.13425   
5         0.27575  0.21075  0.27050  0.5545  0.56800  0.39025  0.12300   

                                    ...                                     \
tagId       220      240      258   ...     972      982     992      999    
movieId                             ...                                      
1        0.78025  0.35550  0.28625  ...  0.75000  0.37550  0.4345  0.49000   
2        0.35625  0.11600  0.27550  ...  0.31650  0.18225  0.1260  0.44075   
3        0.18975  0.23050  0.23075  ...  0.28875  0.17350  0.4220  0.26675   
4        0.18825  0.13475  0.17200  ...  0.33050  0.17175  0.1985  0.18425   
5        0.18025  0.21200  0.17975  ...  0.27225  0.14450  0.3450  0.23850   

                                                               
tagId       1008     1051     1064     1070     1091     1104  
movieId                                                        
1        0.60950  0.36300  0.42050  0.26150  0.41050  0.29925  
2        0.14825  0.15050  0.22075  0.23475  0.19125  0.31600  
3        0.21175  0.20725  0.21875  0.41825  0.21250  0.16800  
4        0.17600  0.18325  0.28400  0.35950  0.18575  0.17850  
5        0.18300  0.15025  0.21400  0.28475  0.18750  0.26975  

[5 rows x 50 columns]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10381 entries, 1 to 131170
Data columns (total 50 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   (relevance, 21)    10381 non-null  float64
 1   (relevance, 22)    10381 non-null  float64
 2   (relevance, 169)   10381 non-null  float64
 3   (relevance, 188)   10381 non-null  float64
 4   (relevance, 195)   10381 non-null  float64
 5   (relevance, 203)   10381 non-null  float64
 6   (relevance, 212)   10381 non-null  float64
 7   (relevance, 220)   10381 non-null  float64
 8   (relevance, 240)   10381 non-null  float64
 9   (relevance, 258)   10381 non-null  float64
 10  (relevance, 270)   10381 non-null  float64
 11  (relevance, 277)   10381 non-null  float64
 12  (relevance, 299)   10381 non-null  float64
 13  (relevance, 302)   10381 non-null  float64
 14  (relevance, 323)   10381 non-null  float64
 15  (relevance, 351)   10381 non-null  float64
 16  (relevance, 378)   10

None

We check that the pivot table genome_top is refleting the mapping of the movies, two alike movies should be with close distance.

In [15]:
#function computed the distance between two movies
def distance_movies(movie1,movie2) :
  return (sum(genome_top.loc[movie1]-genome_top.loc[movie2]))

In [16]:
#distance between Toy Story and Jumaji (different movies)
print(dfmovies.loc[1].title)
print(dfmovies.loc[2].title)

print("distance between the above movies :" ,distance_movies(1,2))

Toy Story (1995)
Jumanji (1995)
distance between the above movies : 6.826249999999998


In [17]:
#distance between two Harry Potter movies (alike movies)
print(dfmovies.loc[4896].title)
print(dfmovies.loc[5816].title)

print("distance between the above movies :", distance_movies(4896,5816))

Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
Harry Potter and the Chamber of Secrets (2002)
distance between the above movies : 0.34250000000000047


The alike movies are very closed wrt two others movies which means we can use a clustering method on the pivot table genome_top to classify the type of movies

Clustering of the movies in function of their tag relevancy

In [18]:
#creation of model clf ky Kmeans method with 100 clusters)
clf = KMeans(n_clusters = 100)

#fitting of the model clf on pivot table
clf.fit(genome_top)



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(


KMeans(n_clusters=100)

In [19]:
#Adding of the cluster labels to our Dataframe genome_top
genome_top["cluster"]=clf.labels_
    


In [20]:
cluster_centroid=clf.cluster_centers_

In [21]:
genome_top.to_csv('../data/genome_cluster.csv')

In [22]:
np.savetxt('../data/cluster_centroid.csv',cluster_centroid,delimiter=';')

Evaluation of the relevancy of the clustering by counting the number of 

occurence of each cluster


In [23]:
#counting the number of occurence of each cluster
cluster_repartition=genome_top.value_counts(subset="cluster")
print(cluster_repartition)

cluster
52    318
43    270
20    261
34    238
41    214
     ... 
3      51
33     51
63     50
19     48
75     48
Length: 100, dtype: int64


The largest cluster is less than 10 times bigger than the smallest so 100 clusters is a good value.



In [24]:
#Adding of the cluster labels to our Dataframe rating too by a merge
dfratings=dfratings.merge(right=genome_top["cluster"],on='movieId',how='left')


In [25]:
dfratings.to_csv('../data/dfratings_cluster.csv')

Exemple with one user, to be remplace by login

In [26]:
user_test=int(1)

extract of the user ratings

In [27]:
seen_by_test=dfratings[dfratings["userId"]==user_test]

In [28]:
print(seen_by_test)

     userId  movieId  rating   timestamp  cluster
0         1        2     3.5  1112486027     61.0
1         1       29     3.5  1112484676     18.0
2         1       32     3.5  1112484819     63.0
3         1       47     3.5  1112484727     16.0
4         1       50     3.5  1112484580     16.0
..      ...      ...     ...         ...      ...
170       1     8507     5.0  1094786027     32.0
171       1     8636     4.5  1112485493     54.0
172       1     8690     3.5  1112484795     94.0
173       1     8961     4.0  1112485629     67.0
174       1    31696     4.0  1112485748     54.0

[175 rows x 5 columns]


top 100 Movies of the users (by ratings)


In [29]:
top_movies_user=seen_by_test.groupby("movieId").mean().sort_values(ascending=False,by="rating").head(100)

In [30]:
print(top_movies_user)

         userId  rating     timestamp  cluster
movieId                                       
8507        1.0     5.0  1.094786e+09     32.0
4993        1.0     5.0  1.112485e+09     57.0
5952        1.0     5.0  1.112485e+09     57.0
7153        1.0     5.0  1.112485e+09     57.0
1196        1.0     4.5  1.112485e+09     57.0
...         ...     ...           ...      ...
8690        1.0     3.5  1.112485e+09     94.0
8482        1.0     3.5  1.112486e+09     99.0
4720        1.0     3.5  1.094786e+09     57.0
5679        1.0     3.5  1.094786e+09     85.0
7045        1.0     3.5  1.112486e+09     79.0

[100 rows x 4 columns]


Repartition of the cluster inside the top 100, and stop if more than 10 différents clusters

In [31]:
cluster_user=top_movies_user.value_counts(subset="cluster", normalize=True).head(5)
print(cluster_user)

cluster
57.0    0.11
30.0    0.07
3.0     0.06
54.0    0.06
16.0    0.06
dtype: float64



We can observe that there are no clusters very frequent wrt the others. All clusters represent less than 10% of the top 100 movies. So we cut after the top 10 clusters.

In [32]:
cluster_user=cluster_user/(cluster_user.sum())//0.20+1.00
print(cluster_user)

cluster
57.0    2.0
30.0    1.0
3.0     1.0
54.0    1.0
16.0    1.0
dtype: float64


We select the movies not seen by the user

In [33]:
not_seen_by_test=dfmovies.merge(right=seen_by_test,on='movieId',how='left')

not_seen_by_test=not_seen_by_test[not_seen_by_test["userId"]!=user_test]


not_seen_by_test.head()


,movieId,title,genres,year,coef_age,userId,rating,timestamp,cluster
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,4.65396,NaN,NaN,NaN,NaN
2,3,Grumpier Old Men (1995),Comedy|Romance,1995,4.65396,NaN,NaN,NaN,NaN
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995,4.65396,NaN,NaN,NaN,NaN
4,5,Father of the Bride Part II (1995),Comedy,1995,4.65396,NaN,NaN,NaN,NaN
5,6,Heat (1995),Action|Crime|Thriller,1995,4.65396,NaN,NaN,NaN,NaN


The following cell checks in a film has been seen by the user (test cells)

In [34]:
seen_by_test.head()
print(seen_by_test[seen_by_test["movieId"]==858])


Empty DataFrame
Columns: [userId, movieId, rating, timestamp, cluster]
Index: []


We drop the useless columns

In [35]:
not_seen_by_test=not_seen_by_test.drop(["userId","rating","timestamp","cluster"], axis=1)
not_seen_by_test.head()


,movieId,title,genres,year,coef_age
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,4.65396
2,3,Grumpier Old Men (1995),Comedy|Romance,1995,4.65396
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995,4.65396
4,5,Father of the Bride Part II (1995),Comedy,1995,4.65396
5,6,Heat (1995),Action|Crime|Thriller,1995,4.65396


Proposition of 5 Movies proportionnaly to the repartition of the top 100 Movies of the users (by ratings). To test the model, the proprosition are made in the list of the film  seen by user, and let compare what is the rating given by the user

In [36]:
    print(dfratings[(dfratings["userId"]==user_test)].describe())


       userId       movieId      rating     timestamp     cluster
count   175.0    175.000000  175.000000  1.750000e+02  174.000000
mean      1.0   3428.537143    3.742857  1.107833e+09   51.873563
std       0.0   3207.448049    0.382284  7.813424e+06   25.410300
min       1.0      2.000000    3.000000  1.094786e+09    3.000000
25%       1.0   1253.500000    3.500000  1.094786e+09   30.000000
50%       1.0   2692.000000    4.000000  1.112485e+09   57.000000
75%       1.0   4986.500000    4.000000  1.112486e+09   73.000000
max       1.0  31696.000000    5.000000  1.112486e+09   99.000000


In [37]:
proposed_movieId=[]
proposed_movie_title=[]
for i in range(5) :
    chosen_cluster=cluster_user.index[i]
    #print(chosen_cluster)

    cluster_movieId=dfratings[(dfratings["userId"]==user_test)&(dfratings["cluster"]==chosen_cluster)]
    print(cluster_movieId.groupby("movieId").mean().sort_values('rating',ascending=False).head(cluster_user.loc[chosen_cluster].astype(int)).index)
    proposed_movieId.append(cluster_movieId.groupby("movieId").mean().sort_values('rating',ascending=False).head(cluster_user.loc[chosen_cluster].astype(int)).index.values)






print(proposed_movieId)
for i in proposed_movieId :
  print(i)
  for j in i :
    print(j)
    proposed_movie_title.append(dfmovies.loc[j].title)
    print(dfratings[(dfratings["userId"]==user_test)&(dfratings["movieId"]==j)])
   
#the ratings are between 4 et 5 si the user liked these movies.

print(proposed_movie_title[0:5])

Int64Index([4993, 5952], dtype='int64', name='movieId')
Int64Index([1215], dtype='int64', name='movieId')
Int64Index([253], dtype='int64', name='movieId')
Int64Index([8636], dtype='int64', name='movieId')
Int64Index([296], dtype='int64', name='movieId')
[array([4993, 5952], dtype=int64), array([1215], dtype=int64), array([253], dtype=int64), array([8636], dtype=int64), array([296], dtype=int64)]
[4993 5952]
4993
     userId  movieId  rating   timestamp  cluster
131       1     4993     5.0  1112484682     57.0
5952
     userId  movieId  rating   timestamp  cluster
142       1     5952     5.0  1112484619     57.0
[1215]
1215
    userId  movieId  rating   timestamp  cluster
36       1     1215     4.0  1094786082     30.0
[253]
253
   userId  movieId  rating   timestamp  cluster
8       1      253     4.0  1112484940      3.0
[8636]
8636
     userId  movieId  rating   timestamp  cluster
171       1     8636     4.5  1112485493     54.0
[296]
296
    userId  movieId  rating   timestamp  

Our model is far better than the mean so the model is validate.

Comparaison with mean rate given by the user to ensure our model is better than the random

Proposition of 5 Movies proportionnaly to the repartition of the top 100 Movies of the users (by ratings).
The proprosition are made in the list of the film not seen by user. The top rating movies are selected, so that is to say "classical" of each type.

In [38]:
proposed_movieId=[]
proposed_movie_title=[]
for i in range(5) :
    chosen_cluster=cluster_user.index[i]
    #print(chosen_cluster)

    cluster_movieId=dfratings[dfratings["cluster"]==chosen_cluster].merge(right=not_seen_by_test,on='movieId',how='inner')
    print(cluster_movieId.groupby("movieId").mean().sort_values('rating',ascending=False).head(cluster_user.loc[chosen_cluster].astype(int)).index)
    proposed_movieId.append(cluster_movieId.groupby("movieId").mean().sort_values('rating',ascending=False).head(cluster_user.loc[chosen_cluster].astype(int)).index.values)


print(proposed_movieId)
for i in proposed_movieId :
  print(i)
  for j in i :
    print(j)
    proposed_movie_title.append(dfmovies.loc[j].title)
    print(dfratings[dfratings["movieId"]==j].groupby("movieId").mean())

proposed_movie_completed=proposed_movie_title[0:5]   
print(proposed_movie_completed)



Int64Index([2571, 79132], dtype='int64', name='movieId')
Int64Index([7075], dtype='int64', name='movieId')
Int64Index([1797], dtype='int64', name='movieId')
Int64Index([91542], dtype='int64', name='movieId')
Int64Index([858], dtype='int64', name='movieId')
[array([ 2571, 79132], dtype=int64), array([7075], dtype=int64), array([1797], dtype=int64), array([91542], dtype=int64), array([858], dtype=int64)]
[ 2571 79132]
2571
               userId    rating     timestamp  cluster
movieId                                               
2571     69092.710504  4.187186  1.152514e+09     57.0
79132
               userId    rating     timestamp  cluster
movieId                                               
79132    69196.991942  4.156172  1.344454e+09     57.0
[7075]
7075
               userId    rating     timestamp  cluster
movieId                                               
7075     69473.817664  3.950142  1.188813e+09     30.0
[1797]
1797
               userId    rating     timestamp  clu

Proposition of 5 Movies proportionnaly to the repartition of the clusters in the top 100 Movies of the users. The proposed Movies are the best rated form each cluster (by ratings) multiplied by a coef to prefer recent movies.
The proprosition are made in the list of the film not seen by user.

In [39]:
proposed_movieId=[]
proposed_movie_title=[]
for i in range(5) :
    chosen_cluster=cluster_user.index[i]
    #print(chosen_cluster)

    cluster_movieId=dfratings[dfratings["cluster"]==chosen_cluster].merge(right=not_seen_by_test,on='movieId',how='inner')
    cluster_movieId["rating_coef_age"]=cluster_movieId["rating"]+20*cluster_movieId["coef_age"]

    print(cluster_movieId.groupby("movieId").mean().sort_values('rating_coef_age',ascending=False).head(cluster_user.loc[chosen_cluster].astype(int)).index)
    proposed_movieId.append(cluster_movieId.groupby("movieId").mean().sort_values('rating_coef_age',ascending=False).head(cluster_user.loc[chosen_cluster].astype(int)).index.values)


print(proposed_movieId)
for i in proposed_movieId :
  print(i)
  for j in i :
    print(j)
    proposed_movie_title.append(dfmovies.loc[j].title)
    print(dfratings[dfratings["movieId"]==j].groupby("movieId").mean())

   
print(proposed_movie_title[0:5])

proposed_movie_completed.extend(proposed_movie_title[0:5])

Int64Index([109487, 91529], dtype='int64', name='movieId')
Int64Index([93422], dtype='int64', name='movieId')
Int64Index([102407], dtype='int64', name='movieId')
Int64Index([110102], dtype='int64', name='movieId')
Int64Index([112556], dtype='int64', name='movieId')
[array([109487,  91529], dtype=int64), array([93422], dtype=int64), array([102407], dtype=int64), array([110102], dtype=int64), array([112556], dtype=int64)]
[109487  91529]
109487
              userId    rating     timestamp  cluster
movieId                                              
109487   70066.87234  4.023864  1.421397e+09     57.0
91529
               userId   rating     timestamp  cluster
movieId                                              
91529    68914.246751  4.00021  1.378670e+09     57.0
[93422]
93422
               userId    rating     timestamp  cluster
movieId                                               
93422    62493.893939  3.780303  1.388383e+09     30.0
[102407]
102407
               userId    rat

In [40]:
print(proposed_movie_completed)

['Matrix, The (1999)', 'Inception (2010)', 'Court Jester, The (1956)', 'Everest (1998)', 'Sherlock Holmes: A Game of Shadows (2011)', 'Interstellar (2014)', 'Dark Knight Rises, The (2012)', 'Starbuck (2011)', 'Great Gatsby, The (2013)', 'Captain America: The Winter Soldier (2014)']


In [41]:
proposed_movie_pivot=pd.DataFrame(proposed_movie_completed)
proposed_movie_pivot.head(10)

,0
0,"Matrix, The (1999)"
1,Inception (2010)
2,"Court Jester, The (1956)"
3,Everest (1998)
4,Sherlock Holmes: A Game of Shadows (2011)
5,Interstellar (2014)
6,"Dark Knight Rises, The (2012)"
7,Starbuck (2011)
8,"Great Gatsby, The (2013)"
9,Captain America: The Winter Soldier (2014)


In [42]:
proposed_movie_pivot.to_csv('../data/proposed_movie_pivot.csv')